## **Setup**

In [1]:
from google.colab import drive
mount_path = '/content/gdrive/'
drive.mount(mount_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [2]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

#hide
!pip install utils
from utils import *

     |████████████████████████████████| 727kB 8.6MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 1.0MB 17.3MB/s 
     |████████████████████████████████| 358kB 39.0MB/s 
     |████████████████████████████████| 40kB 5.6MB/s 
     |████████████████████████████████| 92kB 10.9MB/s 
     |████████████████████████████████| 10.5MB 32.9MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 2.6MB 40.9MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=1.0.0; python_version >= "3.0", but you'll have pandas 1.1.1 which is incompatible.


In [3]:
from fastai.text.all import *

In [4]:
import os
import shutil
import glob
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import pytz
from zipfile import ZipFile
from tempfile import TemporaryDirectory

from sklearn.metrics import f1_score, accuracy_score

# plot options
# plt.rcParams.update({'font.size': 11})
plt.style.use('fivethirtyeight')

In [5]:
# Set and test path to competition data files
competition_path = 'My Drive/AI For Good - AI Blitz 3/FNEWS/Data/'
model_path='My Drive/AI For Good - AI Blitz 3/FNEWS/Models/'
filename = 'train.zip'
assert(os.path.exists(f'{mount_path}/{competition_path}/{filename}'))
print('Drive mounted correctly and data accessible')

Drive mounted correctly and data accessible


# Unzip and combine dataframes
Combine the train and validation sets from the competition organizers, we're going to use our own!


In [6]:
home_dir='/content/All_Data/'

# All_Data is the home directory for the images, from which training and validation splits will be taken
if not os.path.exists(home_dir):
  os.mkdir(home_dir)

  for filename in ['train','val','test']:
    with ZipFile(f'{mount_path}/{competition_path}/{filename}.zip', 'r') as zip_ref:
      zip_ref.extractall(home_dir)
    

In [7]:
combdf=pd.read_csv(f'{home_dir}/train.csv').append(
    pd.read_csv(f'{home_dir}/val.csv'),ignore_index=True)

print(combdf.shape)


(270669, 2)


CV splits

In [8]:
def trainvaldfs(combdf, num_folds, val_pct, randomseed=None):
  '''
  returns a dictionary[fold_num] : dataframe w/ a Boolean column isval indicating the entry is for validation
  eg cvdict[0]
  '''
  cvdict={key:None for key in range(num_folds)}

  assert(val_pct<=(1/num_folds))

  if randomseed is None:
    rng = np.random.RandomState() # random seeding
  else:
    rng = np.random.RandomState(randomseed) # deterministic seeding
  
  inds=combdf.index.to_list()
  # shuffle the image array. NOTE all the randomness in the train-val split comes from this shuffle
  rng.shuffle(inds)

  # size of the fold
  foldsz=np.floor(len(inds)/num_folds).astype(int)
  # how many validation items
  numval=np.floor(val_pct*len(inds)).astype(int)

  for cvfold in range(num_folds):
    outdf=combdf.copy()    
    outdf['isval']=False

    valinds=inds[(cvfold*foldsz):(cvfold*foldsz+numval)]
    outdf.loc[valinds,'isval']=True

    cvdict[cvfold]=outdf.copy()

  return cvdict

In [ ]:
# smalldf=combdf.iloc[:10000]
# cvdict=trainvaldfs(smalldf, num_folds=2, val_pct=0.15, randomseed=42)

# cvdict=trainvaldfs(combdf, num_folds=3, val_pct=0.2, randomseed=42)

# Load in learner and data

In [ ]:
# dls=TextDataLoaders.from_df(cvdict[0], text_col='text', label_col='label', valid_col='isval',
#                         y_block=CategoryBlock, bs=64)

In [ ]:
# learn=text_classifier_learner(dls, AWD_LSTM, metrics=[accuracy, F1Score()], cbs=[ShowGraphCallback()])

In [ ]:
# learn.fine_tune(base_lr=1e-2, freeze_epochs=3, epochs=5)

In [10]:
models=[AWD_QRNN]

for model in models:  
  cvdict=trainvaldfs(combdf, num_folds=3, val_pct=0.2)
  for cvfold in range(3):
    print('-------')
    print(cvfold)
    print('-------')

    dls=TextDataLoaders.from_df(cvdict[cvfold], text_col='text', label_col='label', valid_col='isval',
                        y_block=CategoryBlock, bs=64)
    
    learn=text_classifier_learner(dls, model, metrics=[accuracy, F1Score()], pretrained=False)
    
    learn.fit_one_cycle(5,1e-2)
    learn.fit_one_cycle(5,1e-3)
    learn.fit_one_cycle(5,1e-4)
  
    
    datestr=datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%y%m%d_%H%M")
    learn.export(mount_path+model_path+datestr+'_'+str(model).split('.')[-1][:-2]+'_cv'+str(cvfold)+'.pkl')

-------
0
-------


RuntimeError: ignored

In [ ]:
# # models=[AWD_LSTM,AWD_QRNN]
# models=[AWD_QRNN]

# for model in models:  
#   cvdict=trainvaldfs(combdf, num_folds=3, val_pct=0.2)
#   for cvfold in range(3):
#     print('-------')
#     print(cvfold)
#     print('-------')

#     dls=TextDataLoaders.from_df(cvdict[cvfold], text_col='text', label_col='label', valid_col='isval',
#                         y_block=CategoryBlock, bs=64)
    
#     learn=text_classifier_learner(dls, model, metrics=[accuracy, F1Score()])
    
#     learn.fine_tune(base_lr=1e-2, freeze_epochs=3, epochs=5)

    
#     datestr=datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%y%m%d_%H%M")
#     learn.export(mount_path+model_path+datestr+'_'+str(model).split('.')[-1][:-2]+'_cv'+str(cvfold)+'.pkl')

-------
0
-------


/usr/local/lib/python3.6/dist-packages/fastai/text/learner.py:230: UserWarning: There are no pretrained weights for that architecture yet!
  warn("There are no pretrained weights for that architecture yet!")


RuntimeError: ignored